Erkennung von Distribution Shifts mithilfe des Kolmogorov-Smirnov-Test 

In [4]:
import pandas as pd
from scipy.stats import ks_2samp

# Laden der Daten
file_path = '/Users/ericduhme/Documents/UNI/PAMI/PAMI_Projekt/z_Simulator/test_daten.csv' # Dateipfad anpassen
data = pd.read_csv(file_path)

# Ermitteln der einzigartigen Wochen
weeks = data['Week'].unique()
weeks.sort()

# Ergebnisse speichern
results = []

# Kolmogorov-Smirnov-Test für jede Woche mit der vorhergehenden Woche durchführen
for i in range(1, len(weeks)):
    week_1 = weeks[i-1]
    week_2 = weeks[i]
    
    # Daten der beiden Wochen extrahieren
    data_week_1 = data[data['Week'] == week_1]['Weight']
    data_week_2 = data[data['Week'] == week_2]['Weight']
    
    # Kolmogorov-Smirnov-Test durchführen
    ks_stat, p_value = ks_2samp(data_week_1, data_week_2)
    
    # Ergebnisse speichern
    results.append({
        'Week_1': week_1,
        'Week_2': week_2,
        'KS_Statistic': ks_stat,
        'P_Value': p_value
    })

# Kolmogorov-Smirnov-Test für Woche 1 mit allen anderen Wochen durchführen
week_1 = weeks[0]
data_week_1 = data[data['Week'] == week_1]['Weight']
for i in range(1, len(weeks)):
    week_2 = weeks[i]
    data_week_2 = data[data['Week'] == week_2]['Weight']
    
    # Kolmogorov-Smirnov-Test durchführen
    ks_stat, p_value = ks_2samp(data_week_1, data_week_2)
    
    # Ergebnisse speichern
    results.append({
        'Comparison_Type': 'Week 1 vs Other Weeks',
        'Week_1': week_1,
        'Week_2': week_2,
        'KS_Statistic': ks_stat,
        'P_Value': p_value
    })

# Ergebnisse in einen DataFrame umwandeln und anzeigen
results_df = pd.DataFrame(results)
results_df


,Week_1,Week_2,KS_Statistic,P_Value,Comparison_Type
0,0,1,0.039305,0.170779,NaN
1,1,2,0.031419,0.414356,NaN
2,2,3,0.023476,0.775599,NaN
3,3,4,0.016676,0.979722,NaN
4,0,1,0.039305,0.170779,Week 1 vs Other Weeks
5,0,2,0.053180,0.026451,Week 1 vs Other Weeks
6,0,3,0.045604,0.075262,Week 1 vs Other Weeks
7,0,4,0.038454,0.207902,Week 1 vs Other Weeks
